In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

Using TensorFlow backend.


In [2]:
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

sess = tf.Session()
graph = tf.get_default_graph()

# IMPORTANT: models have to be loaded AFTER SETTING THE SESSION for keras! 
# Otherwise, their weights will be unavailable in the threads after the session there has been set
set_session(sess)

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch>=29000:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(r"C:\Users\rishi\Documents\GAN Images\mnist_%d.png" % epoch)
        plt.close()


In [4]:
global sess
global graph
with graph.as_default():
    set_session(sess)
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, sample_interval=100)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params:

c:\users\rishi\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.450266, acc.: 59.38%] [G loss: 0.565242]
1 [D loss: 0.347410, acc.: 79.69%] [G loss: 0.550451]
2 [D loss: 0.322664, acc.: 90.62%] [G loss: 0.643037]
3 [D loss: 0.318696, acc.: 89.06%] [G loss: 0.787442]
4 [D loss: 0.307242, acc.: 89.06%] [G loss: 0.935471]
5 [D loss: 0.237145, acc.: 98.44%] [G loss: 1.109931]

c:\users\rishi\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'



6 [D loss: 0.218225, acc.: 96.88%] [G loss: 1.422538]
7 [D loss: 0.157318, acc.: 100.00%] [G loss: 1.568770]
8 [D loss: 0.148211, acc.: 100.00%] [G loss: 1.848568]
9 [D loss: 0.131587, acc.: 100.00%] [G loss: 1.998418]
10 [D loss: 0.102357, acc.: 100.00%] [G loss: 2.230104]
11 [D loss: 0.086192, acc.: 100.00%] [G loss: 2.397900]
12 [D loss: 0.063524, acc.: 100.00%] [G loss: 2.560001]
13 [D loss: 0.052083, acc.: 100.00%] [G loss: 2.610697]
14 [D loss: 0.050136, acc.: 100.00%] [G loss: 2.675703]
15 [D loss: 0.041496, acc.: 100.00%] [G loss: 2.897472]
16 [D loss: 0.035531, acc.: 100.00%] [G loss: 2.883403]
17 [D loss: 0.033518, acc.: 100.00%] [G loss: 2.900063]
18 [D loss: 0.042832, acc.: 100.00%] [G loss: 3.059083]
19 [D loss: 0.033181, acc.: 100.00%] [G loss: 3.214345]
20 [D loss: 0.028892, acc.: 100.00%] [G loss: 3.175272]
21 [D loss: 0.019318, acc.: 100.00%] [G loss: 3.452246]
22 [D loss: 0.028069, acc.: 100.00%] [G loss: 3.412022]
23 [D loss: 0.024026, acc.: 100.00%] [G loss: 3.3456

...

29322 [D loss: 0.662582, acc.: 57.81%] [G loss: 1.061787]
29323 [D loss: 0.614957, acc.: 70.31%] [G loss: 1.080607]
29324 [D loss: 0.538270, acc.: 76.56%] [G loss: 0.990114]
29325 [D loss: 0.658203, acc.: 56.25%] [G loss: 0.976530]
29326 [D loss: 0.618814, acc.: 62.50%] [G loss: 1.025271]
29327 [D loss: 0.612917, acc.: 68.75%] [G loss: 1.134470]
29328 [D loss: 0.629699, acc.: 60.94%] [G loss: 0.950158]
29329 [D loss: 0.638264, acc.: 62.50%] [G loss: 1.099592]
29330 [D loss: 0.616031, acc.: 65.62%] [G loss: 1.005900]
29331 [D loss: 0.591171, acc.: 68.75%] [G loss: 1.086708]
29332 [D loss: 0.586476, acc.: 68.75%] [G loss: 0.994385]
29333 [D loss: 0.657932, acc.: 56.25%] [G loss: 0.989302]
29334 [D loss: 0.629719, acc.: 64.06%] [G loss: 1.139793]
29335 [D loss: 0.563503, acc.: 71.88%] [G loss: 1.120046]
29336 [D loss: 0.540286, acc.: 73.44%] [G loss: 1.103415]
29337 [D loss: 0.591277, acc.: 68.75%] [G loss: 1.018841]
29338 [D loss: 0.559887, acc.: 71.88%] [G loss: 0.962119]
29339 [D loss:

29464 [D loss: 0.693710, acc.: 53.12%] [G loss: 1.140854]
29465 [D loss: 0.611848, acc.: 62.50%] [G loss: 1.046922]
29466 [D loss: 0.595004, acc.: 68.75%] [G loss: 0.964199]
29467 [D loss: 0.642921, acc.: 67.19%] [G loss: 1.013294]
29468 [D loss: 0.588971, acc.: 73.44%] [G loss: 0.953734]
29469 [D loss: 0.718024, acc.: 51.56%] [G loss: 1.006043]
29470 [D loss: 0.662126, acc.: 57.81%] [G loss: 1.129461]
29471 [D loss: 0.586709, acc.: 64.06%] [G loss: 0.976089]
29472 [D loss: 0.576633, acc.: 73.44%] [G loss: 1.059658]
29473 [D loss: 0.616922, acc.: 59.38%] [G loss: 1.029264]
29474 [D loss: 0.675275, acc.: 56.25%] [G loss: 1.061105]
29475 [D loss: 0.640583, acc.: 59.38%] [G loss: 0.953418]
29476 [D loss: 0.602727, acc.: 65.62%] [G loss: 0.996434]
29477 [D loss: 0.632564, acc.: 62.50%] [G loss: 1.050883]
29478 [D loss: 0.602091, acc.: 70.31%] [G loss: 1.039467]
29479 [D loss: 0.606618, acc.: 60.94%] [G loss: 1.011657]
29480 [D loss: 0.659824, acc.: 62.50%] [G loss: 0.924674]
29481 [D loss:

29606 [D loss: 0.520302, acc.: 73.44%] [G loss: 0.960318]
29607 [D loss: 0.651521, acc.: 59.38%] [G loss: 0.961339]
29608 [D loss: 0.607688, acc.: 68.75%] [G loss: 0.886086]
29609 [D loss: 0.669100, acc.: 53.12%] [G loss: 0.909791]
29610 [D loss: 0.597414, acc.: 64.06%] [G loss: 0.956699]
29611 [D loss: 0.523534, acc.: 79.69%] [G loss: 0.947197]
29612 [D loss: 0.659153, acc.: 65.62%] [G loss: 1.025490]
29613 [D loss: 0.577405, acc.: 65.62%] [G loss: 0.978487]
29614 [D loss: 0.603619, acc.: 67.19%] [G loss: 0.982222]
29615 [D loss: 0.535824, acc.: 73.44%] [G loss: 0.944196]
29616 [D loss: 0.770296, acc.: 51.56%] [G loss: 1.043180]
29617 [D loss: 0.614017, acc.: 64.06%] [G loss: 0.901132]
29618 [D loss: 0.609368, acc.: 70.31%] [G loss: 1.036995]
29619 [D loss: 0.603575, acc.: 62.50%] [G loss: 1.056497]
29620 [D loss: 0.566102, acc.: 67.19%] [G loss: 1.039264]
29621 [D loss: 0.588123, acc.: 73.44%] [G loss: 1.075614]
29622 [D loss: 0.619136, acc.: 60.94%] [G loss: 1.043333]
29623 [D loss:

29748 [D loss: 0.616405, acc.: 64.06%] [G loss: 1.012545]
29749 [D loss: 0.610627, acc.: 64.06%] [G loss: 0.940637]
29750 [D loss: 0.584783, acc.: 70.31%] [G loss: 1.163022]
29751 [D loss: 0.583288, acc.: 71.88%] [G loss: 1.110414]
29752 [D loss: 0.626500, acc.: 60.94%] [G loss: 0.959425]
29753 [D loss: 0.578370, acc.: 71.88%] [G loss: 1.036269]
29754 [D loss: 0.677876, acc.: 60.94%] [G loss: 0.971790]
29755 [D loss: 0.613672, acc.: 68.75%] [G loss: 1.166384]
29756 [D loss: 0.656442, acc.: 60.94%] [G loss: 0.930982]
29757 [D loss: 0.658292, acc.: 64.06%] [G loss: 0.941575]
29758 [D loss: 0.658812, acc.: 60.94%] [G loss: 1.074126]
29759 [D loss: 0.633172, acc.: 67.19%] [G loss: 1.068143]
29760 [D loss: 0.644750, acc.: 67.19%] [G loss: 1.003925]
29761 [D loss: 0.635769, acc.: 60.94%] [G loss: 1.132668]
29762 [D loss: 0.640648, acc.: 65.62%] [G loss: 1.239371]
29763 [D loss: 0.619088, acc.: 62.50%] [G loss: 1.148229]
29764 [D loss: 0.628060, acc.: 68.75%] [G loss: 1.014193]
29765 [D loss:

29890 [D loss: 0.619298, acc.: 59.38%] [G loss: 1.035988]
29891 [D loss: 0.642011, acc.: 60.94%] [G loss: 1.004253]
29892 [D loss: 0.725005, acc.: 59.38%] [G loss: 1.036442]
29893 [D loss: 0.609906, acc.: 62.50%] [G loss: 1.040951]
29894 [D loss: 0.623959, acc.: 65.62%] [G loss: 1.085573]
29895 [D loss: 0.591102, acc.: 76.56%] [G loss: 0.922413]
29896 [D loss: 0.686291, acc.: 62.50%] [G loss: 0.961505]
29897 [D loss: 0.662949, acc.: 51.56%] [G loss: 0.924331]
29898 [D loss: 0.670716, acc.: 53.12%] [G loss: 0.865721]
29899 [D loss: 0.588784, acc.: 68.75%] [G loss: 0.936515]
29900 [D loss: 0.591785, acc.: 68.75%] [G loss: 0.976420]
29901 [D loss: 0.626899, acc.: 67.19%] [G loss: 1.021386]
29902 [D loss: 0.595220, acc.: 67.19%] [G loss: 1.057887]
29903 [D loss: 0.591627, acc.: 71.88%] [G loss: 1.022565]
29904 [D loss: 0.652971, acc.: 54.69%] [G loss: 1.158455]
29905 [D loss: 0.628211, acc.: 62.50%] [G loss: 1.049413]
29906 [D loss: 0.580307, acc.: 71.88%] [G loss: 1.168001]
29907 [D loss:

In [6]:
def generate_images(cnt):
    c=1
    while (c<=cnt):
        noise = np.random.normal(0, 1, (1, 100))
        gen_imgs = gan.generator.predict(noise)
        gen_imgs = 0.5*gen_imgs + 0.5
        plt.imshow(gen_imgs[0, :, :, 0], cmap='gray')
        plt.axis('off')
        plt.savefig(r"C:\Users\rishi\Documents\GAN Images\generated\gan_{}.jpg".format(c), bbox_inches = 'tight', pad_inches = -0.0005)
        print("Images {} saved.".format(c))
        c+=1
        plt.close()
generate_images(10000)

Images 1 saved.
Images 2 saved.
Images 3 saved.
Images 4 saved.
Images 5 saved.
Images 6 saved.
Images 7 saved.
Images 8 saved.
Images 9 saved.
Images 10 saved.
Images 11 saved.
Images 12 saved.
Images 13 saved.
Images 14 saved.
Images 15 saved.
Images 16 saved.
Images 17 saved.
Images 18 saved.
Images 19 saved.
Images 20 saved.
Images 21 saved.
Images 22 saved.
Images 23 saved.
Images 24 saved.
Images 25 saved.
Images 26 saved.
Images 27 saved.
Images 28 saved.
Images 29 saved.
Images 30 saved.
Images 31 saved.
Images 32 saved.
Images 33 saved.
Images 34 saved.
Images 35 saved.
Images 36 saved.
Images 37 saved.
Images 38 saved.
Images 39 saved.
Images 40 saved.
Images 41 saved.
Images 42 saved.
Images 43 saved.
Images 44 saved.
Images 45 saved.
Images 46 saved.
Images 47 saved.
Images 48 saved.
Images 49 saved.
Images 50 saved.
Images 51 saved.
Images 52 saved.
Images 53 saved.
Images 54 saved.
Images 55 saved.
Images 56 saved.
Images 57 saved.
Images 58 saved.
Images 59 saved.
Images

Images 467 saved.
Images 468 saved.
Images 469 saved.
Images 470 saved.
Images 471 saved.
Images 472 saved.
Images 473 saved.
Images 474 saved.
Images 475 saved.
Images 476 saved.
Images 477 saved.
Images 478 saved.
Images 479 saved.
Images 480 saved.
Images 481 saved.
Images 482 saved.
Images 483 saved.
Images 484 saved.
Images 485 saved.
Images 486 saved.
Images 487 saved.
Images 488 saved.
Images 489 saved.
Images 490 saved.
Images 491 saved.
Images 492 saved.
Images 493 saved.
Images 494 saved.
Images 495 saved.
Images 496 saved.
Images 497 saved.
Images 498 saved.
Images 499 saved.
Images 500 saved.
Images 501 saved.
Images 502 saved.
Images 503 saved.
Images 504 saved.
Images 505 saved.
Images 506 saved.
Images 507 saved.
Images 508 saved.
Images 509 saved.
Images 510 saved.
Images 511 saved.
Images 512 saved.
Images 513 saved.
Images 514 saved.
Images 515 saved.
Images 516 saved.
Images 517 saved.
Images 518 saved.
Images 519 saved.
Images 520 saved.
Images 521 saved.
Images 522

Images 931 saved.
Images 932 saved.
Images 933 saved.
Images 934 saved.
Images 935 saved.
Images 936 saved.
Images 937 saved.
Images 938 saved.
Images 939 saved.
Images 940 saved.
Images 941 saved.
Images 942 saved.
Images 943 saved.
Images 944 saved.
Images 945 saved.
Images 946 saved.
Images 947 saved.
Images 948 saved.
Images 949 saved.
Images 950 saved.
Images 951 saved.
Images 952 saved.
Images 953 saved.
Images 954 saved.
Images 955 saved.
Images 956 saved.
Images 957 saved.
Images 958 saved.
Images 959 saved.
Images 960 saved.
Images 961 saved.
Images 962 saved.
Images 963 saved.
Images 964 saved.
Images 965 saved.
Images 966 saved.
Images 967 saved.
Images 968 saved.
Images 969 saved.
Images 970 saved.
Images 971 saved.
Images 972 saved.
Images 973 saved.
Images 974 saved.
Images 975 saved.
Images 976 saved.
Images 977 saved.
Images 978 saved.
Images 979 saved.
Images 980 saved.
Images 981 saved.
Images 982 saved.
Images 983 saved.
Images 984 saved.
Images 985 saved.
Images 986

Images 1366 saved.
Images 1367 saved.
Images 1368 saved.
Images 1369 saved.
Images 1370 saved.
Images 1371 saved.
Images 1372 saved.
Images 1373 saved.
Images 1374 saved.
Images 1375 saved.
Images 1376 saved.
Images 1377 saved.
Images 1378 saved.
Images 1379 saved.
Images 1380 saved.
Images 1381 saved.
Images 1382 saved.
Images 1383 saved.
Images 1384 saved.
Images 1385 saved.
Images 1386 saved.
Images 1387 saved.
Images 1388 saved.
Images 1389 saved.
Images 1390 saved.
Images 1391 saved.
Images 1392 saved.
Images 1393 saved.
Images 1394 saved.
Images 1395 saved.
Images 1396 saved.
Images 1397 saved.
Images 1398 saved.
Images 1399 saved.
Images 1400 saved.
Images 1401 saved.
Images 1402 saved.
Images 1403 saved.
Images 1404 saved.
Images 1405 saved.
Images 1406 saved.
Images 1407 saved.
Images 1408 saved.
Images 1409 saved.
Images 1410 saved.
Images 1411 saved.
Images 1412 saved.
Images 1413 saved.
Images 1414 saved.
Images 1415 saved.
Images 1416 saved.
Images 1417 saved.
Images 1418 

Images 1802 saved.
Images 1803 saved.
Images 1804 saved.
Images 1805 saved.
Images 1806 saved.
Images 1807 saved.
Images 1808 saved.
Images 1809 saved.
Images 1810 saved.
Images 1811 saved.
Images 1812 saved.
Images 1813 saved.
Images 1814 saved.
Images 1815 saved.
Images 1816 saved.
Images 1817 saved.
Images 1818 saved.
Images 1819 saved.
Images 1820 saved.
Images 1821 saved.
Images 1822 saved.
Images 1823 saved.
Images 1824 saved.
Images 1825 saved.
Images 1826 saved.
Images 1827 saved.
Images 1828 saved.
Images 1829 saved.
Images 1830 saved.
Images 1831 saved.
Images 1832 saved.
Images 1833 saved.
Images 1834 saved.
Images 1835 saved.
Images 1836 saved.
Images 1837 saved.
Images 1838 saved.
Images 1839 saved.
Images 1840 saved.
Images 1841 saved.
Images 1842 saved.
Images 1843 saved.
Images 1844 saved.
Images 1845 saved.
Images 1846 saved.
Images 1847 saved.
Images 1848 saved.
Images 1849 saved.
Images 1850 saved.
Images 1851 saved.
Images 1852 saved.
Images 1853 saved.
Images 1854 

...

Images 7881 saved.
Images 7882 saved.
Images 7883 saved.
Images 7884 saved.
Images 7885 saved.
Images 7886 saved.
Images 7887 saved.
Images 7888 saved.
Images 7889 saved.
Images 7890 saved.
Images 7891 saved.
Images 7892 saved.
Images 7893 saved.
Images 7894 saved.
Images 7895 saved.
Images 7896 saved.
Images 7897 saved.
Images 7898 saved.
Images 7899 saved.
Images 7900 saved.
Images 7901 saved.
Images 7902 saved.
Images 7903 saved.
Images 7904 saved.
Images 7905 saved.
Images 7906 saved.
Images 7907 saved.
Images 7908 saved.
Images 7909 saved.
Images 7910 saved.
Images 7911 saved.
Images 7912 saved.
Images 7913 saved.
Images 7914 saved.
Images 7915 saved.
Images 7916 saved.
Images 7917 saved.
Images 7918 saved.
Images 7919 saved.
Images 7920 saved.
Images 7921 saved.
Images 7922 saved.
Images 7923 saved.
Images 7924 saved.
Images 7925 saved.
Images 7926 saved.
Images 7927 saved.
Images 7928 saved.
Images 7929 saved.
Images 7930 saved.
Images 7931 saved.
Images 7932 saved.
Images 7933 

Images 8318 saved.
Images 8319 saved.
Images 8320 saved.
Images 8321 saved.
Images 8322 saved.
Images 8323 saved.
Images 8324 saved.
Images 8325 saved.
Images 8326 saved.
Images 8327 saved.
Images 8328 saved.
Images 8329 saved.
Images 8330 saved.
Images 8331 saved.
Images 8332 saved.
Images 8333 saved.
Images 8334 saved.
Images 8335 saved.
Images 8336 saved.
Images 8337 saved.
Images 8338 saved.
Images 8339 saved.
Images 8340 saved.
Images 8341 saved.
Images 8342 saved.
Images 8343 saved.
Images 8344 saved.
Images 8345 saved.
Images 8346 saved.
Images 8347 saved.
Images 8348 saved.
Images 8349 saved.
Images 8350 saved.
Images 8351 saved.
Images 8352 saved.
Images 8353 saved.
Images 8354 saved.
Images 8355 saved.
Images 8356 saved.
Images 8357 saved.
Images 8358 saved.
Images 8359 saved.
Images 8360 saved.
Images 8361 saved.
Images 8362 saved.
Images 8363 saved.
Images 8364 saved.
Images 8365 saved.
Images 8366 saved.
Images 8367 saved.
Images 8368 saved.
Images 8369 saved.
Images 8370 

Images 8751 saved.
Images 8752 saved.
Images 8753 saved.
Images 8754 saved.
Images 8755 saved.
Images 8756 saved.
Images 8757 saved.
Images 8758 saved.
Images 8759 saved.
Images 8760 saved.
Images 8761 saved.
Images 8762 saved.
Images 8763 saved.
Images 8764 saved.
Images 8765 saved.
Images 8766 saved.
Images 8767 saved.
Images 8768 saved.
Images 8769 saved.
Images 8770 saved.
Images 8771 saved.
Images 8772 saved.
Images 8773 saved.
Images 8774 saved.
Images 8775 saved.
Images 8776 saved.
Images 8777 saved.
Images 8778 saved.
Images 8779 saved.
Images 8780 saved.
Images 8781 saved.
Images 8782 saved.
Images 8783 saved.
Images 8784 saved.
Images 8785 saved.
Images 8786 saved.
Images 8787 saved.
Images 8788 saved.
Images 8789 saved.
Images 8790 saved.
Images 8791 saved.
Images 8792 saved.
Images 8793 saved.
Images 8794 saved.
Images 8795 saved.
Images 8796 saved.
Images 8797 saved.
Images 8798 saved.
Images 8799 saved.
Images 8800 saved.
Images 8801 saved.
Images 8802 saved.
Images 8803 

Images 9187 saved.
Images 9188 saved.
Images 9189 saved.
Images 9190 saved.
Images 9191 saved.
Images 9192 saved.
Images 9193 saved.
Images 9194 saved.
Images 9195 saved.
Images 9196 saved.
Images 9197 saved.
Images 9198 saved.
Images 9199 saved.
Images 9200 saved.
Images 9201 saved.
Images 9202 saved.
Images 9203 saved.
Images 9204 saved.
Images 9205 saved.
Images 9206 saved.
Images 9207 saved.
Images 9208 saved.
Images 9209 saved.
Images 9210 saved.
Images 9211 saved.
Images 9212 saved.
Images 9213 saved.
Images 9214 saved.
Images 9215 saved.
Images 9216 saved.
Images 9217 saved.
Images 9218 saved.
Images 9219 saved.
Images 9220 saved.
Images 9221 saved.
Images 9222 saved.
Images 9223 saved.
Images 9224 saved.
Images 9225 saved.
Images 9226 saved.
Images 9227 saved.
Images 9228 saved.
Images 9229 saved.
Images 9230 saved.
Images 9231 saved.
Images 9232 saved.
Images 9233 saved.
Images 9234 saved.
Images 9235 saved.
Images 9236 saved.
Images 9237 saved.
Images 9238 saved.
Images 9239 

Images 9620 saved.
Images 9621 saved.
Images 9622 saved.
Images 9623 saved.
Images 9624 saved.
Images 9625 saved.
Images 9626 saved.
Images 9627 saved.
Images 9628 saved.
Images 9629 saved.
Images 9630 saved.
Images 9631 saved.
Images 9632 saved.
Images 9633 saved.
Images 9634 saved.
Images 9635 saved.
Images 9636 saved.
Images 9637 saved.
Images 9638 saved.
Images 9639 saved.
Images 9640 saved.
Images 9641 saved.
Images 9642 saved.
Images 9643 saved.
Images 9644 saved.
Images 9645 saved.
Images 9646 saved.
Images 9647 saved.
Images 9648 saved.
Images 9649 saved.
Images 9650 saved.
Images 9651 saved.
Images 9652 saved.
Images 9653 saved.
Images 9654 saved.
Images 9655 saved.
Images 9656 saved.
Images 9657 saved.
Images 9658 saved.
Images 9659 saved.
Images 9660 saved.
Images 9661 saved.
Images 9662 saved.
Images 9663 saved.
Images 9664 saved.
Images 9665 saved.
Images 9666 saved.
Images 9667 saved.
Images 9668 saved.
Images 9669 saved.
Images 9670 saved.
Images 9671 saved.
Images 9672 

In [7]:
from keras.models import model_from_json
model_json = gan.combined.to_json()
with open(r"C:\Users\rishi\Documents\GAN Images\gan.json", 'w') as json_file:
    json_file.write(model_json)

gan.combined.save_weights("gan.h5")
print("saved model to disk")

saved model to disk
